In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import coco
from torchvision.transforms import ToTensor

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the COCO dataset
dataset = coco(root='path/to/coco/dataset', transform=ToTensor())

# Split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Define the CNN and RNN models
cnn = torchvision.models.resnet50(pretrained=True).to(device)
rnn = nn.LSTM(input_size=2048, hidden_size=512, num_layers=2, batch_first=True).to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(list(cnn.parameters()) + list(rnn.parameters()))

# Define the dataloaders for training and validation
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)
# Training loop
for epoch in range(10):
    for images, captions in train_dataloader:
    # Move data to device
      images = images.to(device)
      captions = captions.to(device)

    # Extract features from images using CNN
    features = cnn(images)

    # Generate dense captions using RNN
    outputs, _ = rnn(features)

    # Compute loss
    loss = criterion(outputs.view(-1, outputs.size(2)), captions.view(-1))

    # Backpropagate and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  # Evaluate on validation set
with torch.no_grad():
    for images, captions in val_dataloader:
      # Move data to device
      images = images.to(device)
      captions = captions.to(device)

      # Extract features from images using CNN
      features = cnn(images)

      # Generate dense captions using RNN
      outputs, _ = rnn(features)

      # Compute loss
      val_loss = criterion(outputs.view(-1, outputs.size(2)), captions.view(-1))
 # Print loss
print(f'Epoch {epoch+1}: train loss = {loss.item():.4f}, val loss = {val_loss.item():.4f}')